In [5]:
import pandas as pd
dfprueba = pd.read_excel("data/prueba modelo cuadrante vehiculo.xlsx")


In [4]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

# Nombre del archivo que subiste
FILE_NAME = "robos_tot_fin.xlsx"

# Paso 1.1: Cargar y limpiar
print("Paso 1: Cargando y limpiando datos...")
df = pd.read_excel(FILE_NAME)

# Asegurar que 'FECHA' sea datetime y extraer el mes y el año
df['FECHA'] = pd.to_datetime(df['FECHA'])
df['AÑO_MES'] = df['FECHA'].dt.to_period('M')

# Asegurar que 'CUADRANTE' sea un tipo de dato consistente (texto o numérico)
df['CUADRANTE'] = df['CUADRANTE'].astype(str)

# Paso 1.2: Agregación por Cuadrante y Mes (nuestra tabla base)
# Contamos el total de robos para cada combinación de AÑO_MES y CUADRANTE
df_base = df.groupby(['AÑO_MES', 'CUADRANTE']).size().reset_index(name='TOTAL_ROBOS')
print(f"Datos agregados. Filas totales: {len(df_base)}")

# Ordenar los datos es crucial para las series de tiempo
df_base = df_base.sort_values(by=['CUADRANTE', 'AÑO_MES']).reset_index(drop=True)

Paso 1: Cargando y limpiando datos...
Datos agregados. Filas totales: 7982


In [5]:
print("Paso 2: Creando características temporales (Lags)...")

# Crear el 'LAG 1' (Robos del mes anterior: T-1)
df_base['ROBOS_T-1'] = df_base.groupby('CUADRANTE')['TOTAL_ROBOS'].shift(1)

# Crear el 'LAG 12' (Robos del mismo mes del año anterior: T-12)
df_base['ROBOS_T-12'] = df_base.groupby('CUADRANTE')['TOTAL_ROBOS'].shift(12)

# Crear la 'Media Móvil' (Promedio de los robos de T-2 a T-6)
# Usamos un 'rolling window' y luego 'shift' para asegurar que solo use datos históricos
df_base['PROMEDIO_T-2_T-6'] = (
    df_base.groupby('CUADRANTE')['TOTAL_ROBOS']
    .rolling(window=5, min_periods=5)
    .mean()
    .reset_index(level=0, drop=True)
    .shift(1) # Aplicamos shift(1) para que el promedio sea de T-2 a T-6
)

Paso 2: Creando características temporales (Lags)...


In [6]:
print("Paso 3A: Creando característica espacial (criminalidad circundante)...")

# Criminalidad Total en la Ciudad (excluyendo el cuadrante actual) en el mes T
df_base['ROBOS_TOTAL_CIUDAD_T'] = df_base.groupby('AÑO_MES')['TOTAL_ROBOS'].transform('sum')
df_base['ROBOS_RESTO_CIUDAD_T'] = df_base['ROBOS_TOTAL_CIUDAD_T'] - df_base['TOTAL_ROBOS']

# Usamos el dato del mes anterior para predecir T+1 (Lag 1)
df_base['ROBOS_VECINOS_T-1'] = df_base.groupby('CUADRANTE')['ROBOS_RESTO_CIUDAD_T'].shift(1)

# *Nota:* Si tienes la matriz de vecinos de Fiscosec, reemplaza este cálculo por la suma
# de los robos solo de los cuadrantes adyacentes.

Paso 3A: Creando característica espacial (criminalidad circundante)...


In [8]:
print("Paso 3B: Creando características contextuales...")

# Extraer el mes y año como características numéricas
df_base['MES'] = df_base['AÑO_MES'].dt.month
df_base['AÑO'] = df_base['AÑO_MES'].dt.year

# Variables sobre el tipo de robo (usando la base original)
# Contamos el % de robos con violencia en el mes anterior (T-1)
# Evitamos KeyError cuando no existe la etiqueta 'SI' en some groups
df_violencia = (
	df.groupby(['AÑO_MES', 'CUADRANTE'])
	  .apply(lambda g: (g['VIOLENCIA'] == 'SI').mean())
	  .reset_index(name='PORC_VIOLENCIA')
	  .sort_values(['CUADRANTE', 'AÑO_MES'])
)
# Shift por CUADRANTE para obtener T-1
df_violencia['PORC_VIOLENCIA'] = df_violencia.groupby('CUADRANTE')['PORC_VIOLENCIA'].shift(1)
df_base = df_base.merge(df_violencia, on=['AÑO_MES', 'CUADRANTE'], how='left')

Paso 3B: Creando características contextuales...


/tmp/ipykernel_65407/3839436039.py:12: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: (g['VIOLENCIA'] == 'SI').mean())


In [9]:
print("Paso 4: Definiendo X, Y y dividiendo datos...")

# Paso 4.1: Definir la Variable Objetivo (Y)
# Movemos 'TOTAL_ROBOS' hacia atrás para que sea el valor a predecir
df_base['Y_ROBOS_T+1'] = df_base.groupby('CUADRANTE')['TOTAL_ROBOS'].shift(-1)

# Paso 4.2: Limpiar filas con valores nulos
# Los primeros 12 meses tendrán NaN debido a los lags T-12 y el promedio
df_modelo = df_base.dropna(subset=['Y_ROBOS_T+1', 'ROBOS_T-12', 'PROMEDIO_T-2_T-6', 'ROBOS_VECINOS_T-1']).copy()
print(f"Filas listas para modelar (sin NaN): {len(df_modelo)}")

# Paso 4.3: Definir X y Y

# X: Todas las variables predictoras
X = df_modelo[[
    'ROBOS_T-1',
    'ROBOS_T-12',
    'PROMEDIO_T-2_T-6',
    'ROBOS_VECINOS_T-1',
    'PORC_VIOLENCIA',
    'MES',
    'AÑO'
]]

# Y: La variable objetivo
Y = df_modelo['Y_ROBOS_T+1']

# Paso 4.4: División de Datos (usando AÑO para una división temporal limpia)
# Entrenar con todo hasta 2024 y probar con 2025 (si tus datos llegan hasta 2025)
X_train = X[df_modelo['AÑO'] < 2025]
Y_train = Y[df_modelo['AÑO'] < 2025]

X_test = X[df_modelo['AÑO'] == 2025]
Y_test = Y[df_modelo['AÑO'] == 2025]

print(f"Tamaño de entrenamiento (hasta 2024): {len(X_train)}")
print(f"Tamaño de prueba (2025): {len(X_test)}")

Paso 4: Definiendo X, Y y dividiendo datos...
Filas listas para modelar (sin NaN): 6996
Tamaño de entrenamiento (hasta 2024): 6648
Tamaño de prueba (2025): 348


In [10]:
print("Paso 5: Entrenamiento del modelo XGBoost...")

# 5.1: Inicializar y entrenar el modelo
modelo_xgb = XGBRegressor(
    objective='reg:squarederror',  # Función para regresión (predicción de un número)
    n_estimators=1000,             # Número de árboles
    learning_rate=0.05,            # Tasa de aprendizaje
    random_state=42,
    tree_method='hist'             # Optimización para datasets grandes
)

modelo_xgb.fit(X_train, Y_train)

# 5.2: Evaluación en el conjunto de prueba
predicciones = modelo_xgb.predict(X_test)

# Asegurar que las predicciones sean números enteros (no puede haber 5.3 robos)
predicciones_int = np.round(predicciones)

# Calcular el Error Cuadrático Medio de la Raíz (RMSE)
# El RMSE se interpreta en la unidad de la variable (Robos). Si te da 2.5, el error promedio es de 2.5 robos.
rmse = np.sqrt(mean_squared_error(Y_test, predicciones_int))

print("\n--- Resultados de la Evaluación ---")
print(f"RMSE (Error Promedio en número de robos): {rmse:.2f} robos")

Paso 5: Entrenamiento del modelo XGBoost...

--- Resultados de la Evaluación ---
RMSE (Error Promedio en número de robos): 1.87 robos


In [11]:
# Importar librerías necesarias (si no se han importado antes)
import pandas as pd
import numpy as np

# --- 1. Filtrar la última observación de cada cuadrante ---
# (Asumiendo que tu df_base ya está ordenado por CUADRANTE y AÑO_MES)

# Selecciona la última fila para cada CUADRANTE, que será la observación más reciente (Octubre 2025)
# Esto ya contiene las variables T-1, T-12, Promedio, etc. calculadas con datos anteriores a Octubre.
datos_futuros = df_base.groupby('CUADRANTE').last().reset_index()

# Eliminar columnas que no son predictoras o que son el objetivo antiguo (Y_ROBOS_T+1)
# Nos quedamos con solo las columnas X que el modelo espera:
datos_futuros = datos_futuros[[
    'CUADRANTE',  # Se mantiene solo para referencia
    'ROBOS_T-1',
    'ROBOS_T-12',
    'PROMEDIO_T-2_T-6',
    'ROBOS_VECINOS_T-1',
    'PORC_VIOLENCIA',
    'MES',
    'AÑO'
]].copy()

# --- 2. Ajustar las variables de Tiempo para la Predicción (T+1) ---

# Queremos predecir NOVIEMBRE 2025 (MES=11, AÑO=2025)

# La columna 'MES' actual es 10 (Octubre), necesitamos cambiarla a 11 (Noviembre)
datos_futuros['MES'] = 11

# La columna 'AÑO' actual es 2025. Se mantiene 2025 porque la predicción es para Noviembre 2025.
datos_futuros['AÑO'] = 2025


# --- 3. Verificar y Preparar para la Predicción ---

# La columna CUADRANTE se elimina para que el DataFrame X sea idéntico al X_train del modelo
X_futuro = datos_futuros.drop(columns=['CUADRANTE'])

print("✅ DataFrame 'datos_futuros' (X_futuro) creado con éxito para predecir Noviembre 2025.")
print(f"Número de cuadrantes a predecir: {len(X_futuro)}")
print("--- Primeras 5 filas de X_futuro (Variables de entrada para el modelo) ---")
print(X_futuro.head())

✅ DataFrame 'datos_futuros' (X_futuro) creado con éxito para predecir Noviembre 2025.
Número de cuadrantes a predecir: 77
--- Primeras 5 filas de X_futuro (Variables de entrada para el modelo) ---
   ROBOS_T-1  ROBOS_T-12  PROMEDIO_T-2_T-6  ROBOS_VECINOS_T-1  PORC_VIOLENCIA  \
0        1.0         1.0               1.2              171.0             0.0   
1        1.0         3.0               1.8              144.0             0.0   
2        3.0         7.0               4.4               97.0             0.0   
3        1.0        10.0               2.6               99.0             0.0   
4        1.0         1.0               3.2               99.0             1.0   

   MES   AÑO  
0   11  2025  
1   11  2025  
2   11  2025  
3   11  2025  
4   11  2025  


In [20]:
# 2. Correr la predicción
predicciones_noviembre = modelo_xgb.predict(X_futuro)

# 3. Redondear a números enteros de robos
robos_predichos_noviembre = np.round(predicciones_noviembre).astype(int)

# 4. Combinar con los cuadrantes
resultado_final = pd.DataFrame({
    'CUADRANTE': datos_futuros['CUADRANTE'],
    'ROBOS_PREDICHOS_NOV': robos_predichos_noviembre
})

print("Predicción completada para Noviembre 2025.")
print(resultado_final)

Predicción completada para Noviembre 2025.
   CUADRANTE  ROBOS_PREDICHOS_NOV
0          1                    1
1         11                    2
2         12                    6
3         13                    5
4         14                    2
..       ...                  ...
72        76                    1
73        77                    6
74        78                    1
75         8                    2
76         9                    3

[77 rows x 2 columns]


In [14]:
# Usa la columna TOTAL_ROBOS de tu DataFrame original 'df_base'
# Esto define la distribución histórica de robos por cuadrante/mes

# Por ejemplo, el 75% de las observaciones históricas de TOTAL_ROBOS cae por debajo de este valor:
umbral_alto = df_base['TOTAL_ROBOS'].quantile(0.75) 

# El 90% de las observaciones históricas cae por debajo de este valor:
umbral_muy_alto = df_base['TOTAL_ROBOS'].quantile(0.90) 

print(f"Umbral Histórico (Percentil 75, Alto): {umbral_alto:.0f} robos/mes")
print(f"Umbral Histórico (Percentil 90, Muy Alto): {umbral_muy_alto:.0f} robos/mes")

Umbral Histórico (Percentil 75, Alto): 6 robos/mes
Umbral Histórico (Percentil 90, Muy Alto): 9 robos/mes


In [15]:
def asignar_riesgo(robos):
    if robos >= umbral_muy_alto:
        return 'MUY ALTO'
    elif robos >= umbral_alto:
        return 'ALTO'
    elif robos > 0:
        return 'MEDIO'
    else:
        return 'BAJO' # Cuadrantes donde se predice 0 robos.

# Aplicar la función a las predicciones
resultado_final['NIVEL_RIESGO'] = resultado_final['ROBOS_PREDICHOS_NOV'].apply(asignar_riesgo)

In [16]:
print("Paso 3A: Calculando Centroides de Cuadrante...")

# 1. Calcular el promedio de Latitud y Longitud para cada CUADRANTE
df_centroides = df.groupby('CUADRANTE')[['LATITUD', 'LONGITUD']].mean().reset_index()
df_centroides.rename(columns={'LATITUD': 'CENTROID_LAT', 'LONGITUD': 'CENTROID_LON'}, inplace=True)

# 2. Asignar los centroides a la tabla base de robos
# Usamos un 'merge' con la columna 'CUADRANTE'
df_base = df_base.merge(df_centroides, on='CUADRANTE', how='left')

Paso 3A: Calculando Centroides de Cuadrante...


In [17]:
print("Paso 3B: Creando característica espacial (criminalidad global)...")

# Criminalidad Total en la Ciudad (excluyendo el cuadrante actual) en el mes T
df_base['ROBOS_TOTAL_CIUDAD_T'] = df_base.groupby('AÑO_MES')['TOTAL_ROBOS'].transform('sum')
df_base['ROBOS_RESTO_CIUDAD_T'] = df_base['ROBOS_TOTAL_CIUDAD_T'] - df_base['TOTAL_ROBOS']

# Usamos el dato del mes anterior (T-1) como predictor para T+1
df_base['ROBOS_VECINOS_T-1'] = df_base.groupby('CUADRANTE')['ROBOS_RESTO_CIUDAD_T'].shift(1)

Paso 3B: Creando característica espacial (criminalidad global)...


In [18]:
print("Paso 3C: Creando características contextuales...")

# 1. Características de Fecha
df_base['MES'] = df_base['AÑO_MES'].dt.month
df_base['AÑO'] = df_base['AÑO_MES'].dt.year

# 2. Porcentaje de Violencia (en el mes T-1)
# 2.1. Contar robos con y sin violencia en la tabla original
df_violencia = df.groupby(['AÑO_MES', 'CUADRANTE'])['VIOLENCIA'].value_counts(normalize=True).unstack(fill_value=0)

# El porcentaje de violencia es la columna 'SI'
if 'SI' in df_violencia.columns:
    df_violencia.rename(columns={'SI': 'PORC_VIOLENCIA_T'}, inplace=True)
else:
    # Caso para cuadrantes sin robos con violencia en un mes
    df_violencia['PORC_VIOLENCIA_T'] = 0 

df_violencia = df_violencia[['PORC_VIOLENCIA_T']].reset_index()

# 2.2. Aplicar el Lag 1 (Shift) para usar el dato del mes anterior (T-1)
df_violencia['PORC_VIOLENCIA_T-1'] = df_violencia.groupby('CUADRANTE')['PORC_VIOLENCIA_T'].shift(1)

# 2.3. Unir a la base de datos principal
df_base = df_base.merge(df_violencia[['AÑO_MES', 'CUADRANTE', 'PORC_VIOLENCIA_T-1']], 
                        on=['AÑO_MES', 'CUADRANTE'], 
                        how='left')

# Rellenar cualquier NaN que pudiera surgir de la unión con 0 (si no hay datos históricos, asumir 0%)
df_base['PORC_VIOLENCIA_T-1'] = df_base['PORC_VIOLENCIA_T-1'].fillna(0)

Paso 3C: Creando características contextuales...
